In [3]:
import os
from sklearn.model_selection import train_test_split
import shutil
import pandas as pd

In [4]:
base_dir = '../'
chainsaw_dir = base_dir+'ESC-50-master/wav_chainsaw_5sec/'
no_chainsaw_dir = base_dir+'ESC-50-master/split_wav/'

result_base_dir = base_dir+'data/'

In [3]:
merge_list = []

chainsaw_filenames = [chainsaw_dir+filename for filename in os.listdir(chainsaw_dir)
                         if filename.endswith('.wav')]
chainsaw_filenames = [filename for filename in chainsaw_filenames
                         if os.path.getsize(filename) == 441078]

x_train_chainsaw, x_test_chainsaw \
    = train_test_split(chainsaw_filenames, test_size = 0.3, random_state = 7)
x_train_chainsaw, x_val_chainsaw \
    = train_test_split(x_train_chainsaw, test_size = 0.3, random_state = 7)
    
merge_list += [(os.path.basename(filename), 1, 'train') for filename in x_train_chainsaw]
merge_list += [(os.path.basename(filename), 1, 'val') for filename in x_val_chainsaw]
merge_list += [(os.path.basename(filename), 1, 'test') for filename in x_test_chainsaw]

os.makedirs(result_base_dir, exist_ok=True)
for filename in chainsaw_filenames:
    shutil.copy(filename, result_base_dir)

no_chainsaw_dirs = [no_chainsaw_dir+dirname for dirname in os.listdir(no_chainsaw_dir)
                       if os.path.isdir(no_chainsaw_dir+dirname)]
for dirname in no_chainsaw_dirs:
    no_chainsaw_filenames = [dirname+'/'+filename for filename in os.listdir(dirname)
                             if filename.endswith('.wav')]
    
    x_train_no_chainsaw, x_test_no_chainsaw \
        = train_test_split(no_chainsaw_filenames, test_size = 0.33, random_state = 7)
    x_train_no_chainsaw, x_val_no_chainsaw \
        = train_test_split(x_train_no_chainsaw, test_size = 0.33, random_state = 7)
        
    merge_list += [(os.path.basename(filename), 0, 'train') for filename in x_train_no_chainsaw]
    merge_list += [(os.path.basename(filename), 0, 'val') for filename in x_val_no_chainsaw]
    merge_list += [(os.path.basename(filename), 0, 'test') for filename in x_test_no_chainsaw]
    
    for filename in no_chainsaw_filenames:
        shutil.copy(filename, result_base_dir)

In [4]:
merge_list[:10]

[('1543_51.14.wav', 1, 'train'),
 ('593_50.04.wav', 1, 'train'),
 ('1469_4.35.wav', 1, 'train'),
 ('359_20.05.wav', 1, 'train'),
 ('396_267.27.wav', 1, 'train'),
 ('1355_498.5.wav', 1, 'train'),
 ('219_11.17.wav', 1, 'train'),
 ('777_54.17.wav', 1, 'train'),
 ('827_10.9.wav', 1, 'train'),
 ('806_10.42.wav', 1, 'train')]

In [5]:
df = pd.DataFrame({
                    'filename':[filename for filename, _, _ in merge_list],
                    'label':[label for _, label, _ in merge_list],
                    'type':[data_type for _, _, data_type in merge_list]
                  })
df = df.sample(frac=1).reset_index(drop=True)
df.head

NameError: name 'merge_list' is not defined

In [6]:
df.to_csv('data_annotations.csv', index=False)

NameError: name 'df' is not defined

In [8]:
df = pd.read_csv('data_annotations.csv')
for idx, filename, label, data_type in df.itertuples():
    result_dir = result_base_dir+data_type+'/'
    os.makedirs(result_dir, exist_ok=True)
    shutil.move(result_base_dir+filename, result_dir)